# Homework 3 -- Cloropleth map of grants in Swiss universities

In [8]:
# imports and all that
import pandas as pd

### Data wrangling

In [65]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size

388592

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns.
We also remove data where location cannot be inferred, i.e. which are missing both the institute and the university

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [66]:
grants = grants[grants.University.notnull()]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

# drop rows where location cannot be inferred
filter_no_location = ~((grants.University == "Nicht zuteilbar - NA") & (grants.Institution.isnull()))
grants = grants[filter_no_location]
grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

print(grants.size)
print(grants.shape)

388592
(48574, 8)
